In [1]:
import gym
import time
import torch
import argparse
import numpy as np

from logger import Logger

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.animation as animation

from copy import deepcopy
from argparse import Namespace

from option_critic import OptionCriticFeatures, OptionCriticConv
from option_critic import critic_loss as critic_loss_fn
from option_critic import actor_loss as actor_loss_fn

from train import run
from logger import Logger
from utils import to_tensor
from fourrooms import Fourrooms
from environment import make_env
from experience_replay import ReplayBuffer

## Parameters

In [2]:
ENV = "ltl_fourrooms"

In [3]:
env, is_atari = make_env("ltl_fourrooms", False)

In [4]:
env.spec.specs[0].ldba.delta

[{(): 0, ('phi',): 1, ('psi',): 0, ('phi', 'psi'): 1},
 {(): 2, ('phi',): 2, ('psi',): 0, ('phi', 'psi'): 1},
 {(): 2, ('phi',): 2, ('psi',): 2, ('phi', 'psi'): 2}]

In [5]:
env.spec.specs[0].ldba.state

0

In [6]:
env.spec.specs[0].ldba.reset()

(0, False)

In [7]:
env.spec.specs[0].ldba.step(('phi',))

(1, True)

In [8]:
env.spec.specs[0].ldba.acc

[{(): [None], ('phi',): [True], ('psi',): [True], ('phi', 'psi'): [True]},
 {(): [None], ('phi',): [None], ('psi',): [True], ('phi', 'psi'): [True]},
 {(): [None], ('phi',): [None], ('psi',): [None], ('phi', 'psi'): [None]}]

## Training

In [9]:
args = Namespace(batch_size=32, cuda=True, entropy_reg=0.01, env='ltl_fourrooms', 
                 epsilon_decay=20000, epsilon_min=0.1, epsilon_start=1.0, exp=None, 
                 frame_skip=4, freeze_interval=200, gamma=0.99, learning_rate=0.0005, 
                 logdir='runs', max_history=10000, max_steps_ep=18000, max_steps_total=4000000, 
                 num_options=4, optimal_eps=0.05, seed=0, switch_goal=True, temp=1, 
                 termination_reg=0.01, update_frequency=4, render=False, name="Standard")
# logger = Logger(
#         logdir=args.logdir, 
#         run_name=f"{OptionCriticFeatures.__name__}-{args.env}-{args.exp}-{time.ctime()}")

run(args)

Current goal {16: ('phi',), 17: ('psi',)}


2023/03/11 04:35:35 PM > ep 1 done. total_steps=1000 | reward=0 | episode_steps=1000 | hours=0.002 | epsilon=0.956
2023/03/11 04:35:39 PM > ep 2 done. total_steps=1936 | reward=1.0 | episode_steps=936 | hours=0.003 | epsilon=0.917
2023/03/11 04:35:40 PM > ep 3 done. total_steps=2397 | reward=8.0 | episode_steps=461 | hours=0.003 | epsilon=0.898
2023/03/11 04:35:44 PM > ep 4 done. total_steps=3397 | reward=10 | episode_steps=1000 | hours=0.004 | epsilon=0.859
2023/03/11 04:35:47 PM > ep 5 done. total_steps=4070 | reward=4.0 | episode_steps=673 | hours=0.005 | epsilon=0.834
2023/03/11 04:35:51 PM > ep 6 done. total_steps=4968 | reward=1.0 | episode_steps=898 | hours=0.006 | epsilon=0.802
2023/03/11 04:35:53 PM > ep 7 done. total_steps=5618 | reward=2.0 | episode_steps=650 | hours=0.007 | epsilon=0.780
2023/03/11 04:35:56 PM > ep 8 done. total_steps=6204 | reward=1.0 | episode_steps=586 | hours=0.008 | epsilon=0.760
2023/03/11 04:36:00 PM > ep 9 done. total_steps=7204 | reward=0 | episode

2023/03/11 04:36:54 PM > ep 71 done. total_steps=20470 | reward=1.0 | episode_steps=190 | hours=0.024 | epsilon=0.423
2023/03/11 04:36:54 PM > ep 72 done. total_steps=20496 | reward=1.0 | episode_steps=26 | hours=0.024 | epsilon=0.423
2023/03/11 04:36:54 PM > ep 73 done. total_steps=20540 | reward=1.0 | episode_steps=44 | hours=0.024 | epsilon=0.422
2023/03/11 04:36:57 PM > ep 74 done. total_steps=21112 | reward=6.0 | episode_steps=572 | hours=0.024 | epsilon=0.413
2023/03/11 04:36:59 PM > ep 75 done. total_steps=21627 | reward=12.0 | episode_steps=515 | hours=0.025 | epsilon=0.405
2023/03/11 04:36:59 PM > ep 76 done. total_steps=21716 | reward=4.0 | episode_steps=89 | hours=0.025 | epsilon=0.404
2023/03/11 04:37:00 PM > ep 77 done. total_steps=21835 | reward=3.0 | episode_steps=119 | hours=0.025 | epsilon=0.402
2023/03/11 04:37:00 PM > ep 78 done. total_steps=21866 | reward=2.0 | episode_steps=31 | hours=0.025 | epsilon=0.402
2023/03/11 04:37:01 PM > ep 79 done. total_steps=22094 | re

2023/03/11 04:37:38 PM > ep 141 done. total_steps=31241 | reward=14.0 | episode_steps=93 | hours=0.036 | epsilon=0.289
2023/03/11 04:37:38 PM > ep 142 done. total_steps=31248 | reward=3.0 | episode_steps=7 | hours=0.036 | epsilon=0.289
2023/03/11 04:37:38 PM > ep 143 done. total_steps=31347 | reward=9.0 | episode_steps=99 | hours=0.036 | epsilon=0.288
2023/03/11 04:37:38 PM > ep 144 done. total_steps=31361 | reward=1.0 | episode_steps=14 | hours=0.036 | epsilon=0.288
2023/03/11 04:37:38 PM > ep 145 done. total_steps=31408 | reward=2.0 | episode_steps=47 | hours=0.036 | epsilon=0.287
2023/03/11 04:37:39 PM > ep 146 done. total_steps=31544 | reward=32.0 | episode_steps=136 | hours=0.036 | epsilon=0.286
2023/03/11 04:37:40 PM > ep 147 done. total_steps=31674 | reward=7.0 | episode_steps=130 | hours=0.036 | epsilon=0.285
2023/03/11 04:37:40 PM > ep 148 done. total_steps=31824 | reward=21.0 | episode_steps=150 | hours=0.037 | epsilon=0.283
2023/03/11 04:37:41 PM > ep 149 done. total_steps=3

2023/03/11 04:38:10 PM > ep 210 done. total_steps=38858 | reward=8.0 | episode_steps=49 | hours=0.045 | epsilon=0.229
2023/03/11 04:38:11 PM > ep 211 done. total_steps=39138 | reward=32.0 | episode_steps=280 | hours=0.045 | epsilon=0.227
2023/03/11 04:38:11 PM > ep 212 done. total_steps=39257 | reward=34.0 | episode_steps=119 | hours=0.045 | epsilon=0.226
2023/03/11 04:38:12 PM > ep 213 done. total_steps=39309 | reward=1.0 | episode_steps=52 | hours=0.045 | epsilon=0.226
2023/03/11 04:38:12 PM > ep 214 done. total_steps=39319 | reward=3.0 | episode_steps=10 | hours=0.045 | epsilon=0.226
2023/03/11 04:38:13 PM > ep 215 done. total_steps=39522 | reward=54.0 | episode_steps=203 | hours=0.045 | epsilon=0.225
2023/03/11 04:38:13 PM > ep 216 done. total_steps=39643 | reward=14.0 | episode_steps=121 | hours=0.046 | epsilon=0.224
2023/03/11 04:38:13 PM > ep 217 done. total_steps=39687 | reward=4.0 | episode_steps=44 | hours=0.046 | epsilon=0.224
2023/03/11 04:38:13 PM > ep 218 done. total_step

...

2023/03/11 04:56:22 PM > ep 1681 done. total_steps=395342 | reward=1.0 | episode_steps=319 | hours=0.348 | epsilon=0.100
2023/03/11 04:56:22 PM > ep 1682 done. total_steps=395519 | reward=129.0 | episode_steps=177 | hours=0.348 | epsilon=0.100
2023/03/11 04:56:23 PM > ep 1683 done. total_steps=395864 | reward=38.0 | episode_steps=345 | hours=0.348 | epsilon=0.100
2023/03/11 04:56:24 PM > ep 1684 done. total_steps=396104 | reward=106.0 | episode_steps=240 | hours=0.349 | epsilon=0.100
2023/03/11 04:56:26 PM > ep 1685 done. total_steps=396653 | reward=339.0 | episode_steps=549 | hours=0.349 | epsilon=0.100
2023/03/11 04:56:26 PM > ep 1686 done. total_steps=397004 | reward=4.0 | episode_steps=351 | hours=0.349 | epsilon=0.100
2023/03/11 04:56:27 PM > ep 1687 done. total_steps=397275 | reward=89.0 | episode_steps=271 | hours=0.350 | epsilon=0.100
2023/03/11 04:56:28 PM > ep 1688 done. total_steps=397385 | reward=42.0 | episode_steps=110 | hours=0.350 | epsilon=0.100
2023/03/11 04:56:29 PM 

2023/03/11 04:57:22 PM > ep 1749 done. total_steps=417293 | reward=200.0 | episode_steps=251 | hours=0.365 | epsilon=0.100
2023/03/11 04:57:22 PM > ep 1750 done. total_steps=417412 | reward=46.0 | episode_steps=119 | hours=0.365 | epsilon=0.100
2023/03/11 04:57:24 PM > ep 1751 done. total_steps=417851 | reward=28.0 | episode_steps=439 | hours=0.365 | epsilon=0.100
2023/03/11 04:57:24 PM > ep 1752 done. total_steps=417902 | reward=26.0 | episode_steps=51 | hours=0.365 | epsilon=0.100
2023/03/11 04:57:24 PM > ep 1753 done. total_steps=418073 | reward=1.0 | episode_steps=171 | hours=0.365 | epsilon=0.100
2023/03/11 04:57:26 PM > ep 1754 done. total_steps=418629 | reward=52.0 | episode_steps=556 | hours=0.366 | epsilon=0.100
2023/03/11 04:57:27 PM > ep 1755 done. total_steps=418957 | reward=1.0 | episode_steps=328 | hours=0.366 | epsilon=0.100
2023/03/11 04:57:27 PM > ep 1756 done. total_steps=419257 | reward=4.0 | episode_steps=300 | hours=0.366 | epsilon=0.100
2023/03/11 04:57:28 PM > ep

2023/03/11 04:58:12 PM > ep 1817 done. total_steps=435661 | reward=6.0 | episode_steps=210 | hours=0.379 | epsilon=0.100
2023/03/11 04:58:12 PM > ep 1818 done. total_steps=435747 | reward=4.0 | episode_steps=86 | hours=0.379 | epsilon=0.100
2023/03/11 04:58:13 PM > ep 1819 done. total_steps=436244 | reward=92.0 | episode_steps=497 | hours=0.379 | epsilon=0.100
2023/03/11 04:58:14 PM > ep 1820 done. total_steps=436380 | reward=2.0 | episode_steps=136 | hours=0.379 | epsilon=0.100
2023/03/11 04:58:14 PM > ep 1821 done. total_steps=436506 | reward=16.0 | episode_steps=126 | hours=0.379 | epsilon=0.100
2023/03/11 04:58:16 PM > ep 1822 done. total_steps=437094 | reward=60.0 | episode_steps=588 | hours=0.380 | epsilon=0.100
2023/03/11 04:58:16 PM > ep 1823 done. total_steps=437142 | reward=8.0 | episode_steps=48 | hours=0.380 | epsilon=0.100
2023/03/11 04:58:18 PM > ep 1824 done. total_steps=437850 | reward=117.0 | episode_steps=708 | hours=0.380 | epsilon=0.100
2023/03/11 04:58:18 PM > ep 1

2023/03/11 05:00:03 PM > ep 1885 done. total_steps=476598 | reward=0 | episode_steps=1000 | hours=0.410 | epsilon=0.100
2023/03/11 05:00:03 PM > ep 1886 done. total_steps=476612 | reward=2.0 | episode_steps=14 | hours=0.410 | epsilon=0.100
2023/03/11 05:00:04 PM > ep 1887 done. total_steps=476894 | reward=2.0 | episode_steps=282 | hours=0.410 | epsilon=0.100
2023/03/11 05:00:07 PM > ep 1888 done. total_steps=477894 | reward=0 | episode_steps=1000 | hours=0.411 | epsilon=0.100
2023/03/11 05:00:08 PM > ep 1889 done. total_steps=478272 | reward=3.0 | episode_steps=378 | hours=0.411 | epsilon=0.100
2023/03/11 05:00:10 PM > ep 1890 done. total_steps=478969 | reward=1.0 | episode_steps=697 | hours=0.411 | epsilon=0.100
2023/03/11 05:00:13 PM > ep 1891 done. total_steps=479969 | reward=0 | episode_steps=1000 | hours=0.412 | epsilon=0.100
2023/03/11 05:00:15 PM > ep 1892 done. total_steps=480761 | reward=1.0 | episode_steps=792 | hours=0.413 | epsilon=0.100
2023/03/11 05:00:18 PM > ep 1893 don

2023/03/11 05:01:53 PM > ep 1953 done. total_steps=516769 | reward=5.0 | episode_steps=386 | hours=0.440 | epsilon=0.100
2023/03/11 05:01:54 PM > ep 1954 done. total_steps=517330 | reward=1.0 | episode_steps=561 | hours=0.440 | epsilon=0.100
2023/03/11 05:01:55 PM > ep 1955 done. total_steps=517688 | reward=1.0 | episode_steps=358 | hours=0.441 | epsilon=0.100
2023/03/11 05:01:58 PM > ep 1956 done. total_steps=518688 | reward=0 | episode_steps=1000 | hours=0.441 | epsilon=0.100
2023/03/11 05:02:01 PM > ep 1957 done. total_steps=519688 | reward=0 | episode_steps=1000 | hours=0.442 | epsilon=0.100
2023/03/11 05:02:01 PM > ep 1958 done. total_steps=519734 | reward=1.0 | episode_steps=46 | hours=0.442 | epsilon=0.100
2023/03/11 05:02:01 PM > ep 1959 done. total_steps=519804 | reward=2.0 | episode_steps=70 | hours=0.442 | epsilon=0.100
2023/03/11 05:02:04 PM > ep 1960 done. total_steps=520723 | reward=2.0 | episode_steps=919 | hours=0.443 | epsilon=0.100
2023/03/11 05:02:05 PM > ep 1961 don

## Loading

In [5]:
args = Namespace(batch_size=32, cuda=True, entropy_reg=0.01, env=ENV, 
                 epsilon_decay=20000, epsilon_min=0.1, epsilon_start=1.0, exp=None, 
                 frame_skip=4, freeze_interval=200, gamma=0.99, learning_rate=0.0005, 
                 logdir='runs', max_history=10000, max_steps_ep=18000, max_steps_total=4000000, 
                 num_options=4, optimal_eps=0.05, seed=0, switch_goal=True, temp=1, 
                 termination_reg=0.01, update_frequency=4, render=False, name="Not LTL")


## Instantiate Env
env, is_atari = make_env(ENV, 0, True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
devide='cpu'
## Instantiate Model
option_critic = OptionCriticFeatures(
        in_features=env.observation_space.shape[0],
        num_actions=env.action_space.n,
        num_options=args.num_options,
        temperature=args.temp,
        eps_start=0,
        eps_min=0,
        eps_decay=0,
        eps_test=0,
        device=device
    )

## Load model params
params = torch.load("models/Standard_4_0_1k", map_location=torch.device('cpu'))
option_critic.load_state_dict(params["model_params"])
option_critic.eval()

## Load Env goal
env.init_states = list(env.tocell.keys())
env.goal = {}
env.switch_goal(params["goal_state"])
print("Goal State: {}".format(params["goal_state"]))

Goal State: 62


In [6]:
params["goal_state"]

62

## Policy Graphs

In [16]:
env_states = 104
env_spec = 3
            
for i in range(env_spec):
    
    fig, ax = plt.subplots(figsize=(15, 15))    
#     ax.set_title('State {}'.format(i))
    
    # Plot Grid
    matrice = ax.matshow(env.occupancy, vmin=0, vmax=1)

    # Annulus
    if ENV=="ltl_fourrooms":
        goal_pos = [list(env.tocell[key]) for key in env.goal]
    else:
        goal_pos = [list(env.tocell[env.goal])]
    
    for pos in goal_pos:
        pos.reverse()
    for pos in goal_pos:
        annulus = patches.Annulus(pos, r=0.25, width=0.01, color='red')
        ax.add_patch(annulus)
    
    for j in range(env_states):
        env_obs = torch.zeros(env_states)
        spec_obs[i] = 1
        
        if ENV=="ltl_fourrooms":
            spec_obs = torch.zeros(env_spec)
            env_obs[j] = 1
            obs = torch.concat([env_obs, spec_obs])
        else:
            obs = env_obs
        
        state = option_critic.get_state(to_tensor(obs))
        greedy_option  = option_critic.greedy_option(state)
        action, logp, entropy = option_critic.get_action(state, greedy_option)
        
        pos = list(env.tocell[j])
        pos.reverse()
        
        dir_ = env.directions[action] 
        
        arrow = patches.Arrow(pos[0], pos[1], dir_[0]*0.3, dir_[1]*0.3, width=0.25, color='green')
        ax.add_patch(arrow)
        if ENV=="ltl_fourrooms":
            goal = {env.goal[key][0]: env.tocell[key]  for key in env.goal}
            ax.set_title("{}. {}".format(i, goal))

    plt.show()

In [ ]:
env.

In [13]:
goal_pos

[[9, 7]]

## Run

In [ ]:
## Instantiate Env
env, is_atari = make_env("fourrooms", True)
device = torch.device('cuda' if torch.cuda.is_available() and args.cuda else 'cpu')
device='cpu'
## Instantiate Model
option_critic = OptionCriticFeatures(
        in_features=env.observation_space.shape[0],
        num_actions=env.action_space.n,
        num_options=args.num_options,
        temperature=args.temp,
        eps_start=0,
        eps_min=0,
        eps_decay=0,
        eps_test=0,
        device=device
    )

## Load model params
params = torch.load("models/Standard_4_0_1k", map_location=torch.device('cpu'))
option_critic.load_state_dict(params["model_params"])
option_critic.eval()

## Load Env goal
env.init_states = list(env.tocell.keys())
env.goal = {}
env.switch_goal(params["goal_state"])

obs   = env.reset()
state = option_critic.get_state(to_tensor(obs))
current_option = option_critic.greedy_option(state)
rewards = 0
while True:        
    
    action, logp, entropy = option_critic.get_action(state, current_option)
    next_obs, reward, done, _ = env.step(action)
    rewards += reward
    
    state = option_critic.get_state(to_tensor(next_obs))
    option_termination, current_option = option_critic.predict_option_termination(state, current_option)

    if done:
        obs   = env.reset()
        print(rewards)
        rewards = 0
#         break

#     logger.log_data(reward)

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
